## CURSO DE CIÊNCIA DE DADOS

## TRABALHO 1

## Importando as Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from scipy import stats
from scipy.stats import norm, kstest
%matplotlib inline

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

## Conectando com o site

In [3]:
site_soccer = "https://fbref.com/pt/comps/Big5/stats/jogadores/Maiores-5-Ligas-Europeias-Estatisticas"
gsw = urlopen(site_soccer)

In [4]:
soup = BeautifulSoup(gsw, 'lxml')
type(soup)

bs4.BeautifulSoup

In [5]:
titulo = soup.title
print(titulo)

<title>2022-2023 Maiores 5 Ligas Europeias estatísticas (jogadores) | FBref.com</title>


## Começando a construir o DataSet

In [6]:
# Inspecionando a página descobri que as linhas da tabela que quero pegar estão na tag tr.
# usei find_all('tr') para pegar as linhas com a tag tr.
rows = soup.find_all('tr')
print(rows[:10])

[<tr class="over_header">
<th aria-label="" class="over_header center" colspan="8" data-stat=""></th>
<th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_playing">Tempo de jogo</th>
<th aria-label="" class="over_header center group_start" colspan="8" data-stat="header_performance">Desempenho</th>
<th aria-label="" class="over_header center group_start" colspan="4" data-stat="header_expected">Esperado</th>
<th aria-label="" class="over_header center group_start" colspan="3" data-stat="header_progression">Progressão</th>
<th aria-label="" class="over_header center group_start" colspan="10" data-stat="header_per90">A cada 90 minutos</th><th></th>
</tr>, <tr>
<th aria-label="Class." class="tooltip ranker poptip sort_default_asc show_partial_when_sorting center" data-stat="ranker" data-tip="&lt;strong&gt;Classificação&lt;/strong&gt;&lt;br&gt;Essa é a contagem de linhas, de cima para baixo.&lt;br&gt;Ela é recalculada seguindo a organização de uma coluna." s

In [7]:
# Inspecionando ainda mais a página, descobri que as células da página estavam na tag 'td',
# então rodei as linhas 'tr' e fui pegando todas as linhas 'td' que estavam dentro das linhas 'tr'.
for row in rows:
    row_td = row.find_all('td')
print(row_td)
type(row_td)

[<td class="left" csk="Đuričić Filip" data-append-csv="eccbe56a" data-stat="player"><a href="/pt/jogadores/eccbe56a/Filip-Djuricic">Filip Đuričić</a></td>, <td class="left poptip" data-stat="nationality" data-tip="NT"><a href="/pt/pais/SRB/Serbia-Futebol"><span style="white-space: nowrap"><span class="f-i f-rs" style="">rs</span> SRB</span></a></td>, <td class="center" csk="3.4" data-stat="position">LT,AT</td>, <td class="left" data-stat="team"><a href="/pt/equipes/8ff9e3b3/Sampdoria-Estatisticas">Sampdoria</a></td>, <td class="left" data-stat="comp_level"><span title="Italy"><span class="f-i f-it" style="">it</span></span> <a href="/pt/comps/11/Serie-A-Estatisticas">Serie A</a></td>, <td class="center" data-stat="age">31-132</td>, <td class="center" data-stat="birth_year">1992</td>, <td class="right group_start" data-stat="games">32</td>, <td class="right" data-stat="games_starts">27</td>, <td class="right" csk="2170" data-stat="minutes">2.170</td>, <td class="right" data-stat="minute

bs4.element.ResultSet

In [8]:
# Como pode ser visto no resultado anterior, as linhas ainda estão com descrições 'html' embutidas ,então temos que fazer um 
# clean nas linhas para retirar essa tag 'html'.
str_cells = str(row_td)
text_clean = BeautifulSoup(str_cells, "lxml").get_text()
print(text_clean)

[Filip Đuričić, rs SRB, LT,AT, Sampdoria, it Serie A, 31-132, 1992, 32, 27, 2.170, 24.1, 3, 0, 3, 3, 0, 0, 9, 0, 2.7, 2.7, 1.9, 4,6, 39, 77, 83, 0,12, 0,00, 0,12, 0,12, 0,12, 0,11, 0,08, 0,19, 0,11, 0,19, Partidas]


In [9]:
import re
# Aqui usei todos os passos anteriores para pegar todas as informações necessárias e como em uma list_rows.
list_rows = []
for row in rows:
    cells = row.find_all('td')
    str_cells = str(cells)
    limpar = re.compile('<.*?>')
    limpar2 = (re.sub(limpar, '',str_cells))
    list_rows.append(limpar2)
print(limpar2)
type(limpar2)

[Filip Đuričić, rs SRB, LT,AT, Sampdoria, it Serie A, 31-132, 1992, 32, 27, 2.170, 24.1, 3, 0, 3, 3, 0, 0, 9, 0, 2.7, 2.7, 1.9, 4,6, 39, 77, 83, 0,12, 0,00, 0,12, 0,12, 0,12, 0,11, 0,08, 0,19, 0,11, 0,19, Partidas]


str

In [10]:
#Aqui usei essa list_rows para formar uma dataset.
df_soccer = pd.DataFrame(list_rows)
df_soccer.head(10)

,0
0,[]
1,[]
2,"[Brenden Aaronson, us USA, LT,AT, Leeds United..."
3,"[Paxten Aaronson, us USA, LT,ZG, Eint Frankfur..."
4,"[James Abankwah, ie IRL, ZG, Udinese, it Serie..."
5,"[George Abbott, eng ENG, LT, Tottenham, eng Pr..."
6,"[Yunis Abdelhamid, ma MAR, ZG, Reims, fr Ligue..."
7,"[Himad Abdelli, fr FRA, LT,AT, Angers, fr Ligu..."
8,"[Salis Abdul Samed, gh GHA, LT, Lens, fr Ligue..."
9,"[Laurent Abergel, fr FRA, LT, Lorient, fr Ligu..."


In [11]:
# Aqui usei .str.split para separar os dados em diferentes colunas quando encontrasse um ', '.
# Esse espaço depois da virgula é importante, pois na tabela existem alguns dados que tem casas decimais.
df1_soccer = df_soccer[0].str.split(', ', expand=True)
df1_soccer.head(100)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,[],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,[],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,[Brenden Aaronson,us USA,"LT,AT",Leeds United,eng Premier League,22-232,2000,36,28,2.372,...,"0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31",Partidas]
3,[Paxten Aaronson,us USA,"LT,ZG",Eint Frankfurt,de Bundesliga,19-289,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11",Partidas]
4,[James Abankwah,ie IRL,ZG,Udinese,it Serie A,19-146,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,[Selim Amallah,ma MAR,"LT,AT",Valladolid,es La Liga,26-208,1996,7,3,267,...,"0,34","1,01","0,67","1,01","0,29","0,09","0,38","0,29","0,38",Partidas]
96,[Daniel Amartey,gh GHA,ZG,Leicester City,eng Premier League,28-172,1994,20,18,1.610,...,"0,00","0,00","0,00","0,00","0,06","0,03","0,09","0,06","0,09",Partidas]
97,[Amath,sn SEN,"LT,AT",Mallorca,es La Liga,26-330,1996,28,12,1.227,...,"0,00","0,15","0,15","0,15","0,20","0,06","0,25","0,20","0,25",Partidas]
98,[Jordan Amavi,fr FRA,ZG,Getafe,es La Liga,29-094,1994,5,1,200,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]


## Fazendo a limpeza do Banco de Dados

In [12]:
# Aqui usei str.strip('[') para retirar o caractere '[' da tabela.
df1_soccer[0] = df1_soccer[0].str.strip('[')
df1_soccer.head(100)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,],None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Brenden Aaronson,us USA,"LT,AT",Leeds United,eng Premier League,22-232,2000,36,28,2.372,...,"0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31",Partidas]
3,Paxten Aaronson,us USA,"LT,ZG",Eint Frankfurt,de Bundesliga,19-289,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11",Partidas]
4,James Abankwah,ie IRL,ZG,Udinese,it Serie A,19-146,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Selim Amallah,ma MAR,"LT,AT",Valladolid,es La Liga,26-208,1996,7,3,267,...,"0,34","1,01","0,67","1,01","0,29","0,09","0,38","0,29","0,38",Partidas]
96,Daniel Amartey,gh GHA,ZG,Leicester City,eng Premier League,28-172,1994,20,18,1.610,...,"0,00","0,00","0,00","0,00","0,06","0,03","0,09","0,06","0,09",Partidas]
97,Amath,sn SEN,"LT,AT",Mallorca,es La Liga,26-330,1996,28,12,1.227,...,"0,00","0,15","0,15","0,15","0,20","0,06","0,25","0,20","0,25",Partidas]
98,Jordan Amavi,fr FRA,ZG,Getafe,es La Liga,29-094,1994,5,1,200,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]


In [13]:
# Aqui usei str.strip('[') para retirar o caractere ']' da tabela.
df1_soccer[0] = df1_soccer[0].str.strip(']')
df1_soccer.head(3000)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Brenden Aaronson,us USA,"LT,AT",Leeds United,eng Premier League,22-232,2000,36,28,2.372,...,"0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31",Partidas]
3,Paxten Aaronson,us USA,"LT,ZG",Eint Frankfurt,de Bundesliga,19-289,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11",Partidas]
4,James Abankwah,ie IRL,ZG,Udinese,it Serie A,19-146,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Nadir Zortea,it ITA,ZG,Sassuolo,it Serie A,23-357,1999,10,7,657,...,"0,14","0,14","0,00","0,14","0,02","0,09","0,11","0,02","0,11",Partidas]
2996,Tanguy Zoukrou,fr FRA,ZG,Troyes,fr Ligue 1,20-035,2003,8,5,511,...,"0,00","0,00","0,00","0,00","0,00","0,01","0,01","0,00","0,01",Partidas]
2997,Kurt Zouma,fr FRA,ZG,West Ham,eng Premier League,28-227,1994,25,24,1.989,...,"0,00","0,09","0,09","0,09","0,10","0,00","0,10","0,10","0,10",Partidas]
2998,Petar Zovko,ba BIH,G,Spezia,it Serie A,21-078,2002,1,0,74,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]


In [14]:
# Aqui usei dropna(axis=0, how='any') para retirar as linhas com dados nulos.
df2_soccer = df1_soccer.dropna(axis=0, how='any')
df2_soccer.head(2900)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
2,Brenden Aaronson,us USA,"LT,AT",Leeds United,eng Premier League,22-232,2000,36,28,2.372,...,"0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31",Partidas]
3,Paxten Aaronson,us USA,"LT,ZG",Eint Frankfurt,de Bundesliga,19-289,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11",Partidas]
4,James Abankwah,ie IRL,ZG,Udinese,it Serie A,19-146,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]
5,George Abbott,eng ENG,LT,Tottenham,eng Premier League,17-298,2005,1,0,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]
6,Yunis Abdelhamid,ma MAR,ZG,Reims,fr Ligue 1,35-256,1987,37,37,3.330,...,"0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09",Partidas]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,Szymon Żurkowski,pl POL,LT,Fiorentina,it Serie A,25-259,1997,2,0,32,...,"2,81","2,81","0,00","2,81","0,33","0,06","0,40","0,33","0,40",Partidas]
3002,Szymon Żurkowski,pl POL,LT,Spezia,it Serie A,25-259,1997,10,3,425,...,"0,00","0,00","0,00","0,00","0,12","0,00","0,12","0,12","0,12",Partidas]
3003,Martin Ødegaard,no NOR,LT,Arsenal,eng Premier League,24-176,1998,37,37,3.127,...,"0,20","0,63","0,43","0,63","0,29","0,23","0,52","0,29","0,52",Partidas]
3004,Milan Đurić,ba BIH,AT,Hellas Verona,it Serie A,33-020,1990,28,14,1.366,...,"0,07","0,13","0,07","0,13","0,09","0,08","0,18","0,09","0,18",Partidas]


In [15]:
# Usei .columns para criar o cabeçalho das colunas , ou seja, o que cada coluna se referia.
df3_soccer = pd.DataFrame(df2_soccer)
df3_soccer.columns = ["Jogador", "Nacionalidade", "Posição","Clube" , "Liga" , "Idade", "Ano em que nasceu", "Jogos em que entrou", "Jogos Iniciados" , "Minutos jogados", "90 min jogados", "Gols", "Assistências", "G+A", "Gols Normais", "Penâltis Batidos", "Penâltis Tentados", "Cartões Amarelos", "Cartões Vermelhos", "Gols esperados", "Gols normais previstos", "Gols Assistidos Esperados", "Gols esperados,exceto pênaltis,+gols assistidos", "Carregada Progressiva", "Passes Progressivos", "Passes Progressivos Recebidos", "Gols/90min", "Assis/90min", "(Gols+Assis)/90min", "Gols Normais/90min", "(Gols normais+Assis)/90min" , "Gols previstos por 90min", "Gols Assistidos esperados em 90min", " Gols esperados+gols previstos em 90min", "Gols normais previstos em 90min", "Gols esperados,exceto pênaltis,+gols assistidos em 90min", "Partidas"]
df3_soccer.head(2820)

,Jogador,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,...,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min",Partidas
2,Brenden Aaronson,us USA,"LT,AT",Leeds United,eng Premier League,22-232,2000,36,28,2.372,...,"0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31",Partidas]
3,Paxten Aaronson,us USA,"LT,ZG",Eint Frankfurt,de Bundesliga,19-289,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11",Partidas]
4,James Abankwah,ie IRL,ZG,Udinese,it Serie A,19-146,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]
5,George Abbott,eng ENG,LT,Tottenham,eng Premier League,17-298,2005,1,0,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",Partidas]
6,Yunis Abdelhamid,ma MAR,ZG,Reims,fr Ligue 1,35-256,1987,37,37,3.330,...,"0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09",Partidas]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2929,Hannes Wolf,at AUT,AT,M'Gladbach,de Bundesliga,24-056,1999,18,4,449,...,"0,20","0,40","0,20","0,40","0,29","0,05","0,34","0,29","0,34",Partidas]
2930,Marius Wolf,de GER,ZG,Dortmund,de Bundesliga,28-015,1995,25,17,1.570,...,"0,06","0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23",Partidas]
2931,Jeong Woo-yeong,kr KOR,"LT,AT",Freiburg,de Bundesliga,23-264,1999,26,4,653,...,"0,14","0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46",Partidas]
2932,Chris Wood,nz NZL,AT,Newcastle Utd,eng Premier League,31-186,1991,18,4,446,...,"0,00","0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27",Partidas]


In [16]:
# Usei .drop('Partidas', inplace=True, axis=1) para remover a coluna Partida do meu dataset.
df3_soccer.drop('Partidas', inplace=True, axis=1)
df3_soccer.head(2821)

,Jogador,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden Aaronson,us USA,"LT,AT",Leeds United,eng Premier League,22-232,2000,36,28,2.372,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten Aaronson,us USA,"LT,ZG",Eint Frankfurt,de Bundesliga,19-289,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James Abankwah,ie IRL,ZG,Udinese,it Serie A,19-146,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George Abbott,eng ENG,LT,Tottenham,eng Premier League,17-298,2005,1,0,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis Abdelhamid,ma MAR,ZG,Reims,fr Ligue 1,35-256,1987,37,37,3.330,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,Marius Wolf,de GER,ZG,Dortmund,de Bundesliga,28-015,1995,25,17,1.570,...,"0,06","0,06","0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23"
2931,Jeong Woo-yeong,kr KOR,"LT,AT",Freiburg,de Bundesliga,23-264,1999,26,4,653,...,"0,14","0,14","0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46"
2932,Chris Wood,nz NZL,AT,Newcastle Utd,eng Premier League,31-186,1991,18,4,446,...,"0,40","0,00","0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27"
2933,Chris Wood,nz NZL,AT,Nott'ham Forest,eng Premier League,31-186,1991,7,5,372,...,"0,24","0,00","0,24","0,24","0,24","0,23","0,05","0,28","0,23","0,28"


In [17]:
# Usei .str[2:] para remover os dois primerios caracteres que tinham na tabela Nacionalidade, pois quando trouxe os dados da
# página, as bandeirinhas que tinham na tabela da página onde eu peguei os dados se transformaram em caracteres.
df3_soccer['Nacionalidade'] = df3_soccer['Nacionalidade'].str[2:]
df3_soccer.head(2821)

,Jogador,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden Aaronson,USA,"LT,AT",Leeds United,eng Premier League,22-232,2000,36,28,2.372,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten Aaronson,USA,"LT,ZG",Eint Frankfurt,de Bundesliga,19-289,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James Abankwah,IRL,ZG,Udinese,it Serie A,19-146,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George Abbott,g ENG,LT,Tottenham,eng Premier League,17-298,2005,1,0,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis Abdelhamid,MAR,ZG,Reims,fr Ligue 1,35-256,1987,37,37,3.330,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,Marius Wolf,GER,ZG,Dortmund,de Bundesliga,28-015,1995,25,17,1.570,...,"0,06","0,06","0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23"
2931,Jeong Woo-yeong,KOR,"LT,AT",Freiburg,de Bundesliga,23-264,1999,26,4,653,...,"0,14","0,14","0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46"
2932,Chris Wood,NZL,AT,Newcastle Utd,eng Premier League,31-186,1991,18,4,446,...,"0,40","0,00","0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27"
2933,Chris Wood,NZL,AT,Nott'ham Forest,eng Premier League,31-186,1991,7,5,372,...,"0,24","0,00","0,24","0,24","0,24","0,23","0,05","0,28","0,23","0,28"


In [18]:
# Usei .str[:2] para guardar apenas os dois primeiros caracteres/números que tinham nessa coluna , pois no site depois da idade
# tinham '-***' , sendo * um número, que se referiam aos dias de vida da pessoa.
df3_soccer['Idade'] = df3_soccer['Idade'].str[:2]
df3_soccer.head(2821)

,Jogador,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden Aaronson,USA,"LT,AT",Leeds United,eng Premier League,22,2000,36,28,2.372,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten Aaronson,USA,"LT,ZG",Eint Frankfurt,de Bundesliga,19,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James Abankwah,IRL,ZG,Udinese,it Serie A,19,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George Abbott,g ENG,LT,Tottenham,eng Premier League,17,2005,1,0,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis Abdelhamid,MAR,ZG,Reims,fr Ligue 1,35,1987,37,37,3.330,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,Marius Wolf,GER,ZG,Dortmund,de Bundesliga,28,1995,25,17,1.570,...,"0,06","0,06","0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23"
2931,Jeong Woo-yeong,KOR,"LT,AT",Freiburg,de Bundesliga,23,1999,26,4,653,...,"0,14","0,14","0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46"
2932,Chris Wood,NZL,AT,Newcastle Utd,eng Premier League,31,1991,18,4,446,...,"0,40","0,00","0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27"
2933,Chris Wood,NZL,AT,Nott'ham Forest,eng Premier League,31,1991,7,5,372,...,"0,24","0,00","0,24","0,24","0,24","0,23","0,05","0,28","0,23","0,28"


In [19]:
# Também  tinha uma bandeirinha na coluna Liga que se referia de qual pais essa liga é , decidi retirar esses caracteres
# que acabaram sendo formados.
df3_soccer['Liga'] = df3_soccer['Liga'].str[2:]
df3_soccer.head(2821)

,Jogador,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden Aaronson,USA,"LT,AT",Leeds United,g Premier League,22,2000,36,28,2.372,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George Abbott,g ENG,LT,Tottenham,g Premier League,17,2005,1,0,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,3.330,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,Marius Wolf,GER,ZG,Dortmund,Bundesliga,28,1995,25,17,1.570,...,"0,06","0,06","0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23"
2931,Jeong Woo-yeong,KOR,"LT,AT",Freiburg,Bundesliga,23,1999,26,4,653,...,"0,14","0,14","0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46"
2932,Chris Wood,NZL,AT,Newcastle Utd,g Premier League,31,1991,18,4,446,...,"0,40","0,00","0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27"
2933,Chris Wood,NZL,AT,Nott'ham Forest,g Premier League,31,1991,7,5,372,...,"0,24","0,00","0,24","0,24","0,24","0,23","0,05","0,28","0,23","0,28"


In [20]:
# Algumas bandeirinhas acabaram criando 3 caracteres em vez de 2, aqui estou tirando a letra 'g' que acabou sendo deixado
# pela bandeirinha da inglaterra.
df3_soccer['Liga'] = df3_soccer['Liga'].str.replace('g ', '')
df3_soccer.head(2821)

,Jogador,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden Aaronson,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,2.372,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,173,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George Abbott,g ENG,LT,Tottenham,Premier League,17,2005,1,0,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,3.330,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,Marius Wolf,GER,ZG,Dortmund,Bundesliga,28,1995,25,17,1.570,...,"0,06","0,06","0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23"
2931,Jeong Woo-yeong,KOR,"LT,AT",Freiburg,Bundesliga,23,1999,26,4,653,...,"0,14","0,14","0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46"
2932,Chris Wood,NZL,AT,Newcastle Utd,Premier League,31,1991,18,4,446,...,"0,40","0,00","0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27"
2933,Chris Wood,NZL,AT,Nott'ham Forest,Premier League,31,1991,7,5,372,...,"0,24","0,00","0,24","0,24","0,24","0,23","0,05","0,28","0,23","0,28"


In [21]:
# Estou usando .str.split(' ', 1, expand=True) para separar o Nome do jogador em Nome e Sobrenome.
df3_soccer[['Nome', 'Sobrenome']] = df3_soccer['Jogador'].str.split(' ', 1, expand=True)
df3_soccer.head(2821)

,Jogador,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,...,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min",Nome,Sobrenome
2,Brenden Aaronson,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,2.372,...,"0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31",Brenden,Aaronson
3,Paxten Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,173,...,"0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11",Paxten,Aaronson
4,James Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,63,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",James,Abankwah
5,George Abbott,g ENG,LT,Tottenham,Premier League,17,2005,1,0,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",George,Abbott
6,Yunis Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,3.330,...,"0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09",Yunis,Abdelhamid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,Marius Wolf,GER,ZG,Dortmund,Bundesliga,28,1995,25,17,1.570,...,"0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23",Marius,Wolf
2931,Jeong Woo-yeong,KOR,"LT,AT",Freiburg,Bundesliga,23,1999,26,4,653,...,"0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46",Jeong,Woo-yeong
2932,Chris Wood,NZL,AT,Newcastle Utd,Premier League,31,1991,18,4,446,...,"0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27",Chris,Wood
2933,Chris Wood,NZL,AT,Nott'ham Forest,Premier League,31,1991,7,5,372,...,"0,24","0,24","0,24","0,23","0,05","0,28","0,23","0,28",Chris,Wood


In [22]:
# Usei drop('Jogador', inplace=True, axis=1) para eliminar a coluna Jogador.
df3_soccer.drop('Jogador', inplace=True, axis=1)
df3_soccer.head(2821)

,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,90 min jogados,...,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min",Nome,Sobrenome
2,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,2.372,26.4,...,"0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31",Brenden,Aaronson
3,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,173,1.9,...,"0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11",Paxten,Aaronson
4,IRL,ZG,Udinese,Serie A,19,2004,2,1,63,0.7,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",James,Abankwah
5,g ENG,LT,Tottenham,Premier League,17,2005,1,0,1,0.0,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00",George,Abbott
6,MAR,ZG,Reims,Ligue 1,35,1987,37,37,3.330,37.0,...,"0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09",Yunis,Abdelhamid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,GER,ZG,Dortmund,Bundesliga,28,1995,25,17,1.570,17.4,...,"0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23",Marius,Wolf
2931,KOR,"LT,AT",Freiburg,Bundesliga,23,1999,26,4,653,7.3,...,"0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46",Jeong,Woo-yeong
2932,NZL,AT,Newcastle Utd,Premier League,31,1991,18,4,446,5.0,...,"0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27",Chris,Wood
2933,NZL,AT,Nott'ham Forest,Premier League,31,1991,7,5,372,4.1,...,"0,24","0,24","0,24","0,23","0,05","0,28","0,23","0,28",Chris,Wood


In [23]:
# Aqui criei uma nova ordem nas colunas , pois quero que Nome e Sobrenome sejam as primeiras colunas aparecer.
nova_ordem = ["Nome", "Sobrenome", "Nacionalidade", "Posição","Clube" , "Liga" , "Idade", "Ano em que nasceu", "Jogos em que entrou", "Jogos Iniciados" , "Minutos jogados", "90 min jogados", "Gols", "Assistências", "G+A", "Gols Normais", "Penâltis Batidos", "Penâltis Tentados", "Cartões Amarelos", "Cartões Vermelhos", "Gols esperados", "Gols normais previstos", "Gols Assistidos Esperados", "Gols esperados,exceto pênaltis,+gols assistidos", "Carregada Progressiva", "Passes Progressivos", "Passes Progressivos Recebidos", "Gols/90min", "Assis/90min", "(Gols+Assis)/90min", "Gols Normais/90min", "(Gols normais+Assis)/90min" , "Gols previstos por 90min", "Gols Assistidos esperados em 90min", " Gols esperados+gols previstos em 90min", "Gols normais previstos em 90min", "Gols esperados,exceto pênaltis,+gols assistidos em 90min"]
df3_soccer = df3_soccer.reindex(columns=nova_ordem)
df3_soccer.head(2821)

,Nome,Sobrenome,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden,Aaronson,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten,Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James,Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George,Abbott,g ENG,LT,Tottenham,Premier League,17,2005,1,0,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis,Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2930,Marius,Wolf,GER,ZG,Dortmund,Bundesliga,28,1995,25,17,...,"0,06","0,06","0,11","0,06","0,11","0,07","0,16","0,23","0,07","0,23"
2931,Jeong,Woo-yeong,KOR,"LT,AT",Freiburg,Bundesliga,23,1999,26,4,...,"0,14","0,14","0,28","0,14","0,28","0,29","0,17","0,46","0,29","0,46"
2932,Chris,Wood,NZL,AT,Newcastle Utd,Premier League,31,1991,18,4,...,"0,40","0,00","0,40","0,20","0,20","0,37","0,06","0,43","0,21","0,27"
2933,Chris,Wood,NZL,AT,Nott'ham Forest,Premier League,31,1991,7,5,...,"0,24","0,00","0,24","0,24","0,24","0,23","0,05","0,28","0,23","0,28"


In [24]:
# Usei dropna(axis=0, how='any') para remover novamente as linhas com dados nulos, dessa vez para a coluna
# Sobrenome e Nacionalidade.
df3_soccer = df3_soccer.dropna(subset=['Sobrenome'])
df3_soccer = df3_soccer.dropna(subset=['Nacionalidade'])
df3_soccer.head(2821)

,Nome,Sobrenome,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden,Aaronson,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten,Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James,Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George,Abbott,g ENG,LT,Tottenham,Premier League,17,2005,1,0,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis,Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,Szymon,Żurkowski,POL,LT,Fiorentina,Serie A,25,1997,2,0,...,"0,00","2,81","2,81","0,00","2,81","0,33","0,06","0,40","0,33","0,40"
3002,Szymon,Żurkowski,POL,LT,Spezia,Serie A,25,1997,10,3,...,"0,00","0,00","0,00","0,00","0,00","0,12","0,00","0,12","0,12","0,12"
3003,Martin,Ødegaard,NOR,LT,Arsenal,Premier League,24,1998,37,37,...,"0,43","0,20","0,63","0,43","0,63","0,29","0,23","0,52","0,29","0,52"
3004,Milan,Đurić,BIH,AT,Hellas Verona,Serie A,33,1990,28,14,...,"0,07","0,07","0,13","0,07","0,13","0,09","0,08","0,18","0,09","0,18"


In [25]:
# Removendo todos os dados nulos
df3_soccer.dropna(inplace=True)
df3_soccer.head(2821)

,Nome,Sobrenome,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden,Aaronson,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten,Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James,Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George,Abbott,g ENG,LT,Tottenham,Premier League,17,2005,1,0,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis,Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,Szymon,Żurkowski,POL,LT,Fiorentina,Serie A,25,1997,2,0,...,"0,00","2,81","2,81","0,00","2,81","0,33","0,06","0,40","0,33","0,40"
3002,Szymon,Żurkowski,POL,LT,Spezia,Serie A,25,1997,10,3,...,"0,00","0,00","0,00","0,00","0,00","0,12","0,00","0,12","0,12","0,12"
3003,Martin,Ødegaard,NOR,LT,Arsenal,Premier League,24,1998,37,37,...,"0,43","0,20","0,63","0,43","0,63","0,29","0,23","0,52","0,29","0,52"
3004,Milan,Đurić,BIH,AT,Hellas Verona,Serie A,33,1990,28,14,...,"0,07","0,07","0,13","0,07","0,13","0,09","0,08","0,18","0,09","0,18"


In [26]:
# Aqui tive o mesmo problema que tive anterior na coluna Liga, usei str.replace('g ', '') para remover o caractere g.
df3_soccer['Nacionalidade'] = df3_soccer['Nacionalidade'].str.replace('g ', '')
df3_soccer.head(2821)

,Nome,Sobrenome,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden,Aaronson,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten,Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James,Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George,Abbott,ENG,LT,Tottenham,Premier League,17,2005,1,0,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis,Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,Szymon,Żurkowski,POL,LT,Fiorentina,Serie A,25,1997,2,0,...,"0,00","2,81","2,81","0,00","2,81","0,33","0,06","0,40","0,33","0,40"
3002,Szymon,Żurkowski,POL,LT,Spezia,Serie A,25,1997,10,3,...,"0,00","0,00","0,00","0,00","0,00","0,12","0,00","0,12","0,12","0,12"
3003,Martin,Ødegaard,NOR,LT,Arsenal,Premier League,24,1998,37,37,...,"0,43","0,20","0,63","0,43","0,63","0,29","0,23","0,52","0,29","0,52"
3004,Milan,Đurić,BIH,AT,Hellas Verona,Serie A,33,1990,28,14,...,"0,07","0,07","0,13","0,07","0,13","0,09","0,08","0,18","0,09","0,18"


In [27]:
# Checando se outras bandeirinhas acabaram criando 3 caracteres, descobri que existia mais três que criavam 3 caracteres
#, como anteriormente ja tinha removido 2 caracteres, só precisei retirar o outro caractere extra.
df3_soccer['Nacionalidade'] = df3_soccer['Nacionalidade'].str.replace('r ', '')
df3_soccer['Nacionalidade'] = df3_soccer['Nacionalidade'].str.replace('t ', '')
df3_soccer['Nacionalidade'] = df3_soccer['Nacionalidade'].str.replace('s ', '')
df3_soccer.head(2821)

,Nome,Sobrenome,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
2,Brenden,Aaronson,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
3,Paxten,Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
4,James,Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,George,Abbott,ENG,LT,Tottenham,Premier League,17,2005,1,0,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
6,Yunis,Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3001,Szymon,Żurkowski,POL,LT,Fiorentina,Serie A,25,1997,2,0,...,"0,00","2,81","2,81","0,00","2,81","0,33","0,06","0,40","0,33","0,40"
3002,Szymon,Żurkowski,POL,LT,Spezia,Serie A,25,1997,10,3,...,"0,00","0,00","0,00","0,00","0,00","0,12","0,00","0,12","0,12","0,12"
3003,Martin,Ødegaard,NOR,LT,Arsenal,Premier League,24,1998,37,37,...,"0,43","0,20","0,63","0,43","0,63","0,29","0,23","0,52","0,29","0,52"
3004,Milan,Đurić,BIH,AT,Hellas Verona,Serie A,33,1990,28,14,...,"0,07","0,07","0,13","0,07","0,13","0,09","0,08","0,18","0,09","0,18"


In [28]:
# Aqui resetei o indice para que a primeira linha tenha o índice começando do um também.
df3_soccer = df3_soccer.reset_index(drop=True)
df3_soccer.index = df3_soccer.index + 1
df3_soccer.head(2821)

,Nome,Sobrenome,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,...,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min,Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
1,Brenden,Aaronson,USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,...,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
2,Paxten,Aaronson,USA,"LT,ZG",Eint Frankfurt,Bundesliga,19,2003,7,0,...,"0,00","0,00","0,00","0,00","0,00","0,09","0,03","0,11","0,09","0,11"
3,James,Abankwah,IRL,ZG,Udinese,Serie A,19,2004,2,1,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
4,George,Abbott,ENG,LT,Tottenham,Premier League,17,2005,1,0,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,Yunis,Abdelhamid,MAR,ZG,Reims,Ligue 1,35,1987,37,37,...,"0,03","0,05","0,08","0,03","0,08","0,06","0,03","0,09","0,06","0,09"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2774,Szymon,Żurkowski,POL,LT,Fiorentina,Serie A,25,1997,2,0,...,"0,00","2,81","2,81","0,00","2,81","0,33","0,06","0,40","0,33","0,40"
2775,Szymon,Żurkowski,POL,LT,Spezia,Serie A,25,1997,10,3,...,"0,00","0,00","0,00","0,00","0,00","0,12","0,00","0,12","0,12","0,12"
2776,Martin,Ødegaard,NOR,LT,Arsenal,Premier League,24,1998,37,37,...,"0,43","0,20","0,63","0,43","0,63","0,29","0,23","0,52","0,29","0,52"
2777,Milan,Đurić,BIH,AT,Hellas Verona,Serie A,33,1990,28,14,...,"0,07","0,07","0,13","0,07","0,13","0,09","0,08","0,18","0,09","0,18"


# Arquivo CSV

Criando o Arquivo CSV

In [166]:
csv_file=df3_soccer.to_csv()
print(csv_file)

,Nome,Sobrenome,Nacionalidade,Posição,Clube,Liga,Idade,Ano em que nasceu,Jogos em que entrou,Jogos Iniciados,Minutos jogados,90 min jogados,Gols,Assistências,G+A,Gols Normais,Penâltis Batidos,Penâltis Tentados,Cartões Amarelos,Cartões Vermelhos,Gols esperados,Gols normais previstos,Gols Assistidos Esperados,"Gols esperados,exceto pênaltis,+gols assistidos",Carregada Progressiva,Passes Progressivos,Passes Progressivos Recebidos,Gols/90min,Assis/90min,(Gols+Assis)/90min,Gols Normais/90min,(Gols normais+Assis)/90min,Gols previstos por 90min,Gols Assistidos esperados em 90min, Gols esperados+gols previstos em 90min,Gols normais previstos em 90min,"Gols esperados,exceto pênaltis,+gols assistidos em 90min"
1,Brenden,Aaronson, USA,"LT,AT",Leeds United,Premier League,22,2000,36,28,2.372,26.4,1,3,4,1,0,0,2,0,3.9,3.9,4.2,"8,1",43.0,86.0,151.0,"0,04","0,11","0,15","0,04","0,15","0,15","0,16","0,31","0,15","0,31"
2,Paxten,Aaronson, USA,"LT,ZG",Eint Frankfurt, Bundesliga,19,2003,7,0,173.0,1.9,0,0,0

Salvando o Arquivo CSV

In [ ]:
#df3_soccer.to_csv('df3_soccer_csv2.csv') 
#df3_soccer.to_csv('C:/Users/Vaio/Desktop/df3_soccer_csv2.csv')